# ECDSA tag verification
## Prepare

To interact with Ethereum network, you need Ethereum node up and running.
You can run both notebook and ganache-cli node emulator by `./start.sh` script.

Or you can run ganache-cli Ethereum emulator in separate terminal
`npx ganache-cli -m "dawn finish orchard pluck festival genuine absorb van bike mirror kiss loop"`
(12 words are the seed passphrase to keep addresses and keys constant)

### Connect to Web3
Now connect to Ethereum provider via web3 RPC

In [63]:
from web3 import Web3
w3 = Web3(Web3.HTTPProvider('http://127.0.0.1:8545'))
w3.eth.defaultAccount = w3.eth.accounts[0]

Check if web3 up and running

In [64]:
w3.eth.getBlock('latest')

AttributeDict({'number': 8,
 'hash': HexBytes('0x93212e01fa51e6f80ab9928da00eb1d36c33b62ecc71c80374001107d492ac9a'),
 'parentHash': HexBytes('0xf8105d8f84a77a6e2ecf8b728c9e99059510703bf83a6e6e44eb7325d2dd63ac'),
 'mixHash': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'),
 'nonce': HexBytes('0x0000000000000000'),
 'sha3Uncles': HexBytes('0x1dcc4de8dec75d7aab85b567b6ccd41ad312451b948a7413f0a142fd40d49347'),
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'transactionsRoo

### Solidity version
Solc compiler has to be installed on your machine. Check solidity version (should match pragma statement in your contract)

In [65]:
import subprocess, re, json
solc = subprocess.Popen(['solc', '--version'], stdout=subprocess.PIPE, stdin=subprocess.PIPE)
solc_output = solc.communicate()

m = re.search(r"Version: ([\w\.\+]+)", str(solc_output[0]))
m.group(1)

'0.5.7+commit.6da8b019.Darwin.appleclang'

### Solidity TagRegistry contract
The contract stores tag registry and verifies the signatures reported by authenticators

In [66]:
contract_source_code = b"""

pragma solidity ^0.5.7;

contract TagRegistry {
    mapping (address => bool) public allowedTagIDs;
    
    function allow(address tagID) public {
        allowedTagIDs[tagID] = true;
    }
    
    function ban(address tagID) public {
        allowedTagIDs[tagID] = false;
    }
    
    function isTagIdAllowed(bytes32 signedData, uint8 v, bytes32 r, bytes32 s) view public returns(bool) {
        address tagId = ecrecover(signedData, v, r, s);
        return allowedTagIDs[tagId];
    }
}

"""

Actually compile and generate ABI

In [67]:
solc = subprocess.Popen(['solc', '--combined-json', 'bin,abi', '-'], stdout=subprocess.PIPE, stdin=subprocess.PIPE)
solc_output = solc.communicate(contract_source_code)

See the contract bytecode

In [68]:
bytecode = json.loads(solc_output[0])['contracts']['<stdin>:TagRegistry']['bin']
bytecode

'608060405234801561001057600080fd5b5061035f806100206000396000f3fe608060405234801561001057600080fd5b506004361061004c5760003560e01c806309d7ae181461005157806397c3ccd8146100b8578063e6213d13146100fc578063ff9913e814610158575b600080fd5b61009e6004803603608081101561006757600080fd5b8101908080359060200190929190803560ff169060200190929190803590602001909291908035906020019092919050505061019c565b604051808215151515815260200191505060405180910390f35b6100fa600480360360208110156100ce57600080fd5b81019080803573ffffffffffffffffffffffffffffffffffffffff16906020019092919050505061025f565b005b61013e6004803603602081101561011257600080fd5b81019080803573ffffffffffffffffffffffffffffffffffffffff1690602001909291905050506102b9565b604051808215151515815260200191505060405180910390f35b61019a6004803603602081101561016e57600080fd5b81019080803573ffffffffffffffffffffffffffffffffffffffff1690602001909291905050506102d9565b005b60008060018686868660405160008152602001604052604051808581526020018460ff1660ff168152602001838152602001828152602

See the contract ABI interfaces

In [69]:
abi = json.loads(solc_output[0])['contracts']['<stdin>:TagRegistry']['abi']
abi

'[{"constant":true,"inputs":[{"name":"signedData","type":"bytes32"},{"name":"v","type":"uint8"},{"name":"r","type":"bytes32"},{"name":"s","type":"bytes32"}],"name":"isTagIdAllowed","outputs":[{"name":"","type":"bool"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"tagID","type":"address"}],"name":"ban","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[{"name":"","type":"address"}],"name":"allowedTagIDs","outputs":[{"name":"","type":"bool"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"tagID","type":"address"}],"name":"allow","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"}]'

### Deploy
Instantiate contract fabric and deploy the contract on the net

In [70]:
TagRegistry = w3.eth.contract(abi=abi, bytecode=bytecode)
tx_hash = TagRegistry.constructor().transact()
tx_hash

HexBytes('0x467fe9c344724782a4595c06dba214781e5ff202260e0f611f3589b55cc6e0a0')

Get Tx receipt (see contractAddress)

In [71]:
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
tx_receipt

AttributeDict({'transactionHash': HexBytes('0x467fe9c344724782a4595c06dba214781e5ff202260e0f611f3589b55cc6e0a0'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0x01d0268c531be10eb2e5c9286abc965fe99ff243ffa785e42226a24bdcac510c'),
 'blockNumber': 9,
 'from': '0x90c08087af274b77516df05952273008fea2c4b9',
 'to': None,
 'gasUsed': 283730,
 'cumulativeGasUsed': 283730,
 'contractAddress': '0xE6af14C5B3F3f069b4387B08d48F345b495aBA39',
 'logs': [],
 'status': 1,
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

Initialize contract instance at given address

In [72]:
tag_registry = w3.eth.contract(
    address=tx_receipt.contractAddress,
    abi=abi,
)
tag_registry

# Manufacturing
Tag is a wireless passive equipment with ECDSA IC on board which:
* stores the secret and never expose it to the outside
* reports its public key
* implements ECDSA SECP256k1 signatures and can sign the 32-bit data object with its secret key and send back the result

In [86]:
from eth_keys import keys

class Tag:
    _priv_key = None # never exposed outside
    pub_key = None
    def __init__(self, priv_key):
        self._priv_key = keys.PrivateKey(priv_key)
        
    def get_pub_key(self):
        return self._priv_key.public_key
    
    def sign(self, data):
        return w3.eth.account.signHash(data, private_key=self._priv_key) 

The manufacturer produces a lot of tags, each with its own random private key.
We take two tags:
allowed_tag - will be registered in the registry as genuine 
banned_tag - another tag will have no records in the registry

In [87]:
#Produce two tags with random secret keys
# took SHA3-256 for better randomization. Can take any hash with 256-bit output
import hashlib, os
m = hashlib.sha256() 
m.update(os.urandom(1024))
allowed_tag = Tag(m.digest())
m = hashlib.sha256()
m.update(os.urandom(1024))
banned_tag = Tag(m.digest())
allowed_tag, banned_tag

(<__main__.Tag at 0x1111c39b0>, <__main__.Tag at 0x111155e10>)

# Provisioning
When the tag gets mounted on the rail, it gets authorized as genuine equipment.
It gets scanned, address gets calculated and stored in the registry contract.

In [89]:
allowed_tag_address = allowed_tag.get_pub_key().to_checksum_address()
tag_registry.functions.allow(allowed_tag_address).transact()

HexBytes('0x5b2571186f52527efaaf2fc929e544be6eda0c502c2f33323aaa06825742c91c')

When blockchain transaction gets mined the record securely stored in the contract.
Allowed tag id is in allowed list.

In [90]:
tag_registry.functions.allowedTagIDs(allowed_tag_address).call()

True

banned_tag has been never provisioned to the registry, so it's disallowed

In [91]:
tag_registry.functions.allowedTagIDs(banned_tag.get_pub_key().to_checksum_address()).call()

False

# Operation
The scanner found the tag and received its ECDSA keypair public key.
It's optional step. It's possible to continuosly broadcast `.sign()` messages

In [95]:
allowed_tag.get_pub_key()

'0x0f2891c17dee221c18dd38e8115f4b508532d92ae4590112e9e0075374ea36068890b2bea9773039e9716a074c6b5cc73a1f1ae31d36b0cecd6b7ca16f188484'

To assure the Tag is not a stub radio transmitter broadcasting the public key copied from another ID, the reader uthenticates it.
Reader generates one-time random number of 32 bytes length.

In [96]:
m = hashlib.sha256()
m.update(os.urandom(1024))
rnd = m.digest()
rnd.hex()

'd8c1ec93c7bfe1f14f973985b8311a377967aac45b6dc7ac0af9ab90565edd47'

and sends the number towards the tag. Tag signs it and responds with its signature

In [97]:
allowed_tag_signature = allowed_tag.sign(rnd)
allowed_tag_signature

AttributeDict({'messageHash': HexBytes('0xd8c1ec93c7bfe1f14f973985b8311a377967aac45b6dc7ac0af9ab90565edd47'),
 'r': 54584104147711485840738378979194865709441348205450212112854859713898738347707,
 's': 9623638295190000776758953574573796208096580937679721874590836570587538216013,
 'v': 28,
 'signature': HexBytes('0x78ad82145833ecf5dc9a40b24ab632c17c8b7f4448621b84c5127034985beebb1546c94790aa3cad1a96ec3b6e5c066c565aa115e570f3b5b425c9f43b79544d1c')})

The receiver verifies the number and its signature via `isTagIdAllowed` method of registry

In [98]:
tag_registry.functions.isTagIdAllowed(
    rnd,
    allowed_tag_signature.v,
    allowed_tag_signature.r.to_bytes(32, byteorder='big'),
    allowed_tag_signature.s.to_bytes(32, byteorder='big')).call()

True

Then reader discovers tag which wasn't allowed or was revoked (banned)

In [82]:
banned_tag.get_pub_key()

'0x13d5e96e615e4b49f049e6632ada397c4afd035e21b68e4cdd16b3ba1e9a90c703bf630a1a930ebc5636287cac7faf22eaa3b0aa5cc96fa980b8e371726681d0'

Reader generates new one-time random number

In [83]:
m = hashlib.sha256()
m.update(os.urandom(1024))
rnd = m.digest()
rnd.hex()

'a5c23e01bf6d053d7b4ffa9a317f582efa8bd3190260ce0807e9cf43425b2c9c'

In [84]:
banned_tag_signature = banned_tag.sign(rnd)
banned_tag_signature

AttributeDict({'messageHash': HexBytes('0xa5c23e01bf6d053d7b4ffa9a317f582efa8bd3190260ce0807e9cf43425b2c9c'),
 'r': 99288274403505527175805562138512902220964099144065001876758772970953950441060,
 's': 14654237600920204680838483846240343007229921057513712403165133102175136442635,
 'v': 27,
 'signature': HexBytes('0xdb832bfd161347357539c05cc04757c265d80a7432a4a226c8edc94ab2812e642066012a6dff93d12fedf0149322cacb8fa9c654d77dec5afaa6c1e8652ae90b1b')})

In [85]:
tag_registry.functions.isTagIdAllowed(
    rnd,
    banned_tag_signature.v,
    banned_tag_signature.r.to_bytes(32, byteorder='big'),
    banned_tag_signature.s.to_bytes(32, byteorder='big')).call()

False